# 7.4 word2vec, fastText를 이용한 일본어 단어의 벡터 표현 구현

- 일본어 단어를 word2vec 또는 fastText를 사용하여 벡터화하는 법을 설명합니다.

※　이 장의 파일은 Ubuntu 환경에서의 동작을 전제로 하고 있습니다. Windows와 같이 문자 코드가 다른 환경에서는 동작에 주의하십시오.

# 7.4 학습 목표

1.	학습된 일본어 word2vec 모델로 단어를 벡터 표현으로 변환할 수 있다
2.	학습된 일본어 fastText 모델로 단어를 벡터 표현으로 변환할 수 있다


# 사전 준비

도서의 지시에 따라, 이 장에서 사용하는 데이터를 준비합니다

pip install gensim


# 1. 문서를 읽고, 띄어쓰기, 데이터 세트 작성까지(7.2와 동일합니다)

전처리 및 띄어쓰기하고, 마지막으로 데이터 세트를 작성하는 부분을 구현합니다


In [1]:
# 단어 분할에는 Mecab＋NEologd를 사용
import MeCab

m_t = MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # 이를 통해 스페이스로 단어가 구분됨
    ret = text.strip().split()  # 스페이스로 구분된 리스트로 변환
    return ret



# 전처리로서 정규화 함수를 정의
import re

def preprocessing_text(text):
    # 줄 바꿈, 반각 스페이스, 전각 스페이스를 삭제
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 숫자를 일률적으로 "0"로 설정
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 숫자

    return text


# 전처리 및 Mecab의 단어 분할을 수행하는 함수를 정의한다
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 전처리 정규화
    ret = tokenizer_mecab(text)  # Mecab 단어 분할

    return ret


In [2]:
import torchtext

# tsv나 csv 데이터를 읽을 때, 읽어들인 내용에 대해 수행할 처리를 정의합니다
# 문장과 라벨을 모두 준비합니다

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


# "data" 폴더에서 각 tsv 파일을 읽어들입니다
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 2. 단어의 벡터화

## 2.1 word2vec

단어를 벡터 표현으로 변환합니다.

TorchText에는 일본어로 학습된 데이터가 없는 것은 아니지만, 정확도가 미묘하므로

도호쿠 대학 乾・岡崎 연구실에서 공개된 Word2Vec의 학습된 벡터를 사용합니다.



In [3]:
# 일본어 fasttext의 학습된 벡터를 다운로드합니다

# 도호쿠 대학 乾・岡崎 연구실: 일본어 Wikipedia 엔티티 벡터

# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/
# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2

In [4]:
# torchtext에서 그대로 읽을 수 없으므로 gensim 라이브러리를 사용하여
# Word2Vec의 format으로 다시 저장합니다

# 사전 설치
# pip install gensim

from gensim.models import KeyedVectors


# 일단 gensim 라이브러리로 읽은 뒤, word2vec의 format으로 저장한다
model = KeyedVectors.load_word2vec_format(
    './data/entity_vector/entity_vector.model.bin', binary=True)

# 저장(10분 정도 시간이 걸립니다)
model.wv.save_word2vec_format('./data/japanese_word2vec_vectors.vec')


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL

In [5]:
# torchtext로 단어 벡터로 읽어들입니다
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='./data/japanese_word2vec_vectors.vec')

# 단어 벡터의 내용을 확인합니다
print("한 단어를 표현하는 차원수: ", japanese_word2vec_vectors.dim)
print("단어수: ", len(japanese_word2vec_vectors.itos))


100%|█████████▉| 1014564/1015474 [01:45<00:00, 9573.88it/s] 

한 단어를 표현하는 차원수:  200
단어수:  1015474


In [6]:
# 벡터화한 버전의 vocabulary를 만듭니다
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# vocabulary의 벡터를 확인합니다
print(TEXT.vocab.vectors.shape)  # 49개의 단어가 200차원의 벡터로 표현됨
TEXT.vocab.vectors


torch.Size([49, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-2.8353,  2.5609, -0.5348,  ...,  0.4602,  1.4669, -2.1255],
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.0448, -0.1304,  0.0329,  ...,  0.0825, -0.1386,  0.0417]])

100%|█████████▉| 1014564/1015474 [02:00<00:00, 9573.88it/s]

In [7]:
# vocabulary의 단어 순서를 확인합니다
TEXT.vocab.stoi


defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             ', ': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'negative': 31,
             'positive': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
             '女王': 39,
             '好き': 

In [8]:
# 姫 - 女性 + 男性 의 벡터가 어느 것과 비슷한지 확인합니다
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# 코사인 유사도를 계산
# dim=0 는 0차원에서 계산하도록 지정
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械学習 tensor(-0.1404)


姫 - 女性 + 男性　를 계산한 결과, 예상대로 王子가 가장 가까운 결과가 되었습니다

## 2.2 fastText

word2vec보다 한 단계 진화된 벡터화 기법인 fastText에 의한 단어의 벡터 표현을 사용합니다.

일본어 학습 모델은 이하의 기사에서 공개된 것을 사용합니다.



In [ ]:
# Qiita: 바로 사용 가능한 단어 내장 벡터의 목록
# https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

# Download Word Vectors
# https://drive.google.com/open?id=0ByFQ96A4DgSPNFdleG1GaHcxQzA

In [10]:
# torchtext로 단어 벡터로서 읽어들입니다
# word2vec와는 달리, 곧바로 읽어들일 수 있습니다
from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='./data/vector_neologd/model.vec')
                                    
# 단어 벡터의 내용을 확인합니다
print("한 단어를 표현하는 차원수: ", japanese_fasttext_vectors.dim)
print("단어수: ", len(japanese_fasttext_vectors.itos))



  0%|          | 0/351122 [00:00<?, ?it/s]Skipping token b'351122' with 1-dimensional vector [b'300']; likely a header

 28%|██▊       | 97950/351122 [00:13<00:38, 6494.52it/s]


 53%|█████▎    | 186013/351122 [00:27<00:25, 6506.48it/s]


 78%|███████▊  | 273895/351122 [00:40<00:11, 6532.98it/s]


100%|█████████▉| 350697/351122 [00:51<00:00, 7133.80it/s]

한 단어를 표현하는 차원수:  300
단어수:  351122


In [11]:
# 벡터화한 버전의 vocabulary를 만듭니다
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# vocabulary의 벡터를 확인합니다
print(TEXT.vocab.vectors.shape)  # 52개의 단어가 300차원의 벡터로 표현됨
TEXT.vocab.vectors

# vocabulary의 단어 순서를 확인합니다
TEXT.vocab.stoi


torch.Size([49, 300])


defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             ', ': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'negative': 31,
             'positive': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
             '女王': 39,
             '好き': 


100%|█████████▉| 350697/351122 [01:08<00:00, 7133.80it/s]

In [12]:
# 姫 - 女性 + 男性 의 벡터가 어느 것과 비슷한지 확인합니다
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# 코사인 유사도를 계산
# dim=0 는 0차원에서 계산하도록 지정
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3650)
王 tensor(0.3461)
王子 tensor(0.5531)
機械学習 tensor(0.0952)


姫 - 女性 + 男性　를 계산한 결과, 예상대로 王子가 가장 가까운 결과가 되었습니다

끝